### Decompose the OHU term into radiative and non-radiative terms.

Radiative: Radiative perturbation at surface from T, Q, albedo, cloud, and CO2

Non-radiative: Surface fluxes

In [1]:
# import statements
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe
from dask.diagnostics import ProgressBar

%matplotlib inline

%config InlineBackend.figure_format = "pdf"

In [2]:
gw = xr.open_dataarray('/dx02/janoski/cesm-LE/output/cam5_gauss_weights.nc')
# create function for taking spatial averages, while weighting for latitude
def gw_mean(ds_in, gw=gw, lat_bound_s = -91, lat_bound_n = 91):
    """
    Use xarray/numpy to calculate spatial average while weighting for latitude.
    
    Keyword arguments:
    ds_in -- Dataset or DataArray to take the average of, ALREADY ZONALLY AVERAGED
    gw -- Array of guassian weights. Should only have latitude dimension.
    lat_bound_s -- float, Southern boundary of area to average
    lat_bound_n -- float, Northern boundary of area to average
    """
    if('lon' in ds_in.coords):
        ds_in = ds_in.mean(dim='lon')
    return (ds_in.sel(lat=slice(lat_bound_s,lat_bound_n)) * gw.sel(lat=slice(lat_bound_s,lat_bound_n))/gw.sel(
        lat=slice(lat_bound_s,lat_bound_n)).sum(dim='lat')).sum(dim='lat')

In [3]:
def preprocess(ds):
    dsnew = ds.copy()
    dsnew['time'] = np.arange(1,731,1)
    return dsnew

In [7]:
sfc = xr.open_mfdataset(
    '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.*.h1_surface_flux_perturbs.nc',
                         preprocess=preprocess,combine='nested',concat_dim='ens')
rad = sfc.dFLNS + sfc.dFSNS
nonrad = sfc.dSHFLX + sfc.dLHFLX

Ta = xr.open_mfdataset(
    '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.*.h1_Ta_rad_perturbs_sfc.nc',
                         preprocess=preprocess,combine='nested',concat_dim='ens')
Ts = xr.open_mfdataset(
    '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.*.h1_Ts_rad_perturbs_sfc.nc',
                         preprocess=preprocess,combine='nested',concat_dim='ens')
Q = xr.open_mfdataset(
    '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.*.h1_Q_rad_perturbs.nc',
                         preprocess=preprocess,combine='nested',concat_dim='ens')
Q = Q.FSNS + Q.FLNS
alb = xr.open_mfdataset(
    '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.*.h1_alb_rad_perturbs.nc',
                         preprocess=preprocess,combine='nested',concat_dim='ens')
clw = xr.open_mfdataset(
    '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.*.h1_cloud_LW_perturb_CAM4_sfc.nc',
                         preprocess=preprocess,combine='nested',concat_dim='ens')
csw = xr.open_mfdataset(
    '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.*.h1_cloud_SW_perturb_CAM4_sfc.nc',
                         preprocess=preprocess,combine='nested',concat_dim='ens')
cld = clw.cloud_LW + csw.cloud_SW

# read in CO2 RF
fin = xr.open_dataset('/dx02/janoski/cesm-LE/strat_adj_rf/CAM4_CAMRT_strat_adj_RF.nc',use_cftime=True)

# we need lat and lon from any CESM-LE file
dummy = xr.open_dataset('/dx02/janoski/cesm-LE/output/b40.1850.cam5-lens.4xCO2.01.01.h1_PS.nc')


# make dataarray to regrid too
ds_out = xr.Dataset({'lat': (['lat'], dummy.lat),
                     'lon': (['lon'], dummy.lon),
                    }
                   )

# make regridder
regridder = xe.Regridder(fin, ds_out, 'bilinear',periodic=True)

CO2SW = regridder(fin.FSNS)
CO2LW = -1 * regridder(fin.FLNS)

CO2_arc = np.tile(gw_mean(-1 * (CO2SW + CO2LW),lat_bound_s=70),(2))
CO2_glb = np.tile(gw_mean(-1 * (CO2SW + CO2LW)),(2))

/home/janoski/miniconda3/envs/esmpy_env/lib/python3.7/site-packages/xesmf/frontend.py:534: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


In [10]:
ohr_glb = gw_mean(sfc.dFsfc).mean(dim='ens')
ohr_arc = gw_mean(sfc.dFsfc,lat_bound_s=70).mean(dim='ens')
rad_glb = gw_mean(rad).mean(dim='ens')
rad_arc = gw_mean(rad,lat_bound_s=70).mean(dim='ens')
nonrad_glb = gw_mean(nonrad).mean(dim='ens')
nonrad_arc = gw_mean(nonrad,lat_bound_s=70).mean(dim='ens')
Ta_glb = gw_mean(Ta.FLNS * -1).mean(dim='ens')
Ta_arc = gw_mean(Ta.FLNS * -1,lat_bound_s=70).mean(dim='ens')
Ts_glb = gw_mean(Ts.FLNS * -1).mean(dim='ens')
Ts_arc = gw_mean(Ts.FLNS * -1,lat_bound_s=70).mean(dim='ens')
Q_glb = gw_mean(Q * -1).mean(dim='ens')
Q_arc = gw_mean(Q * -1,lat_bound_s=70).mean(dim='ens')
alb_glb = gw_mean(alb.FSNS * -1).mean(dim='ens')
alb_arc = gw_mean(alb.FSNS * -1,lat_bound_s=70).mean(dim='ens')
cld_glb = gw_mean(cld * -1).mean(dim='ens')
cld_arc = gw_mean(cld * -1,lat_bound_s=70).mean(dim='ens')
with ProgressBar():
    ohr_glb = ohr_glb.compute()
    ohr_arc = ohr_arc.compute()
    Ta_glb = Ta_glb.compute()
    Ta_arc = Ta_arc.compute()
    Ts_glb = Ts_glb.compute()
    Ts_arc = Ts_arc.compute()
    rad_glb = rad_glb.compute()
    rad_arc = rad_arc.compute()
    nonrad_glb = nonrad_glb.compute()
    nonrad_arc = nonrad_arc.compute()
    Q_glb = Q_glb.compute()
    Q_arc = Q_arc.compute()
    alb_glb = alb_glb.compute()
    alb_arc = alb_arc.compute()
    cld_glb = cld_glb.compute()
    cld_arc = cld_arc.compute()

[########################################] | 100% Completed |  1min 17.7s
[########################################] | 100% Completed | 10.1s
[########################################] | 100% Completed |  2min 50.8s
[########################################] | 100% Completed | 17.6s
[########################################] | 100% Completed |  2min 50.9s
[########################################] | 100% Completed | 17.6s
[########################################] | 100% Completed |  2min 33.6s
[########################################] | 100% Completed |  1min 43.6s
[########################################] | 100% Completed |  2min 34.2s
[########################################] | 100% Completed | 27.9s
[########################################] | 100% Completed |  6min 44.1s
[########################################] | 100% Completed |  6min 46.1s
[#####                                   ] | 13% Completed |  1min 58.6s

/home/janoski/miniconda3/envs/esmpy_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed |  2min 50.4s
[########################################] | 100% Completed | 25.7s
[########################################] | 100% Completed |  5min 23.9s
[########################################] | 100% Completed |  5min 20.1s


In [12]:
res_glb = rad_glb - Ta_glb - Ts_glb - Q_glb - alb_glb - cld_glb - CO2_glb
res_arc = rad_arc - Ta_arc - Ts_arc - Q_arc - alb_arc - cld_arc - CO2_arc

In [47]:
fig, ax = plt.subplots()

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

plt.plot(xs,ohr_glb,label='OHR',color='blue')
plt.plot(xs,rad_glb,label='Rad',color='red')
plt.plot(xs,nonrad_glb,label='Non-rad',color='green')

plt.xlim([1,731])
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
plt.ylim([-9,9])
plt.legend(loc='best')
plt.ylabel('Ens. mean ΔF$_{sfc}$ (W/m$^2$), global')
plt.grid()

plt.tight_layout()
plt.show()

<Figure size 432x288 with 1 Axes>

In [44]:
fig, ax = plt.subplots()

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

plt.plot(xs,ohr_arc,label='OHR',color='blue')
plt.plot(xs,rad_arc,label='Rad',color='red')
plt.plot(xs,nonrad_arc,label='Non-rad',color='green')

plt.xlim([1,731])
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
plt.ylim([-45,35])
plt.legend(loc='best')
plt.ylabel('Ens. mean ΔF$_{sfc}$ (W/m$^2$), Arctic')
plt.grid()

plt.tight_layout()
plt.show()

<Figure size 432x288 with 1 Axes>

In [48]:
fig, ax = plt.subplots()

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

plt.plot(xs,rad_glb,label='Rad',color='red')
plt.plot(xs,Ts_glb,label='Ts',color='gold')
plt.plot(xs,Ta_glb,label='Ta',color='forestgreen')
plt.plot(xs,Q_glb,label='Q',color='blueviolet')
plt.plot(xs,alb_glb,label='α',color='deeppink')
plt.plot(xs,cld_glb,label='Cld',color='cyan')
plt.plot(xs,res_glb,label='Res',color='gray')
plt.plot(xs,CO2_glb,label='CO2',color='black')

plt.xlim([1,731])
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
plt.ylim([-9,9])
plt.legend(loc='best',ncol=2)
plt.ylabel('Ens. mean ΔF$_{sfc}$ (W/m$^2$), global')
plt.grid()

plt.tight_layout()
plt.show()

<Figure size 432x288 with 1 Axes>

In [45]:
fig, ax = plt.subplots()

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

plt.plot(xs,rad_arc,label='Rad',color='red')
plt.plot(xs,Ts_arc,label='Ts',color='gold')
plt.plot(xs,Ta_arc,label='Ta',color='forestgreen')
plt.plot(xs,Q_arc,label='Q',color='blueviolet')
plt.plot(xs,alb_arc,label='α',color='magenta')
plt.plot(xs,cld_arc,label='Cld',color='cyan')
plt.plot(xs,res_arc,label='Res',color='gray')
plt.plot(xs,CO2_arc,label='CO2',color='black')

plt.xlim([1,731])
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
plt.ylim([-45,35])
plt.legend(loc='best',ncol=2)
plt.ylabel('Ens. mean ΔF$_{sfc}$ (W/m$^2$), Arctic')
plt.grid()

plt.tight_layout()
plt.show()

<Figure size 432x288 with 1 Axes>